In [1]:
from _shared import *

## Задача 1. Конструирование метрики

Допустим, есть следующие данные о покупках в онлайн-магазине: стоимость (R), себестоимость (C) и факт обращения в поддержку (S). За каждое обращение в поддержку в среднем тратим 110 рублей на оплату работы оператора и компенсацию неудобств покупателям. Хотим максимизировать прибыль. Какую метрику лучше выбрать?

В вариантах ниже приведены формулы, по которым будем вычислять значение для каждой покупки и затем использовать для проверки гипотезы о равенстве средних. Значения переменных R, C и S неотрицательные. S равно 0, если обращения в поддержку не было, иначе равно 1.

In [4]:
answer = 'M = R - С - 110 * S'

## Задача 2. Функция вычисления метрики

In [5]:
"""
Напишите функцию get_metric, которая вычисляет значения метрики из предыдущего задания.

Шаблон решения
"""

import pandas as pd


def get_metric(df, support_cost):
    """Вычисляет значения метрики.

    :param df (pd.DataFrame): таблица с данными о заказах со столбцами:
        - revenue - выручка
        - cost_price - себестоимость
        - support - обращение в поддержку
    :param support_cost (float): средняя стоимость одного обращения в поддержку
    :return (pd.Series): значения метрики
    """
    return df['revenue'] - df['cost_price'] - support_cost * df['support']

In [6]:
df = pd.DataFrame({
    'revenue': [1500, 1800, 2100],
    'cost_price': [1300, 1200, 1600],
    'support': [1, 0, 0],
})
support_cost = 110
metric = get_metric(df, support_cost)
# metric = pd.Series([90, 600, 500])
metric

0     90
1    600
2    500
dtype: int64

In [7]:
# Solution (is the same)
def get_metric(df, support_cost):
    return df['revenue'] - df['cost_price'] - support_cost * df['support']

## Задача 3. Пороговое значение

Определите пороговое значении стоимости обращения в поддержку, при небольшом отклонение от которого меняется результат проверки гипотезы. Другими словами нужно найти значение, при котором p-value совпадает с уровнем значимости и среднее в экспериментальной группе больше, чем в контрольной.

Проверяем гипотезу о равенстве средних для метрики, которую вычисляли в предыдущем задании. В качестве критерия используем тест Стьюдента. Уровень значимости равен 0.05.

Ответ округлите до целого значения.

Код для генерации данных эксперимента

In [8]:
import numpy as np
import pandas as pd

sample_size = 1000

df_control = pd.DataFrame({
    'revenue': [int(np.sin(x / 12) * 600 + 1200) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 12) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 400).astype(int),
})
df_pilot = pd.DataFrame({
    'revenue': [int(np.sin(x / 11 + 1) * 650 + 1250) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 11 + 1) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 300).astype(int),
})